In [2]:
import numpy as np
import cv2
from matplotlib import pyplot as plt
import scipy as sp

def find_if_close(cnt1,cnt2,const):
    M1 = cv2.moments(cnt1)
    M2 = cv2.moments(cnt2)
    
    if(M1["m00"]!=0 and M2["m00"]!=0):
        cX1 = int((M1["m10"] / M1["m00"]))
        cY1 = int((M1["m01"] / M1["m00"]))
        cX2 = int((M2["m10"] / M2["m00"]))
        cY2 = int((M2["m01"] / M2["m00"]))
        
        dist = np.sqrt((cX1-cX2)*(cX1-cX2)+(cY1-cY2)*(cY1-cY2)) 
        if dist < const :
            return True,(cX1,cY1),(cX2,cY2)
    return False,(0,0),(0,0)        
def saveMatInDir(mat,dir,name):
    cv2.imwrite(dir+name, mat)
cap = cv2.VideoCapture('C:\\Users\\dev-user\\Documents\\tennis_videos\\20161208_074126.mp4')
#cap = cv2.VideoCapture('20160519_081947.mp4')

#cap = cv2.VideoCapture("D:\\vidsnew\\vid_last.mp4")
#cap.set(cv2.CAP_PROP_POS_MSEC,161000)
cap.set(cv2.CAP_PROP_POS_MSEC,410000)
fgbg = cv2.createBackgroundSubtractorMOG2(500,160,False)
perFrame=False
orb = cv2.ORB_create()
bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
iii = 0
kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(3,3))
kernel1 = cv2.getStructuringElement(cv2.MORPH_CROSS,(5,5))
oldUnified=[]
unified = []
linearr=[]
counter=0
while(1):
    iii+=1
    ret, frame = cap.read()
    baseframe=frame.copy()
    fgmask = fgbg.apply(frame)
    fgmask = cv2.morphologyEx(fgmask, cv2.MORPH_OPEN, kernel)
    
    black = np.zeros_like(fgmask)
    res=np.zeros_like(frame)
    #res = cv2.bitwise_and(frame,frame,mask = fgmask)
   
    #fgmask = cv2.dilate(fgmask, kernel1, iterations=5)

    ret,thresh = cv2.threshold(fgmask,127,255,cv2.THRESH_BINARY)
    im2, contours, hierarchy = cv2.findContours(thresh,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
    if(len(contours)>100):
        continue
    hsv = cv2.cvtColor(res, cv2.COLOR_BGR2HSV)
    lower_yellow = np.array([20,50,50])
    upper_yellow = np.array([50,255,255])
    maskcol = cv2.inRange(hsv, lower_yellow, upper_yellow)
    res1 = cv2.bitwise_and(res,res,mask = maskcol)
    
    #######
    LENGTH = len(contours)
    status = np.zeros((LENGTH,1))

    for i,cnt1 in enumerate(contours):
        x = i    
        if i != LENGTH-1:
            for j,cnt2 in enumerate(contours[i+1:]):
                x = x+1
                dist,bbbb,bbbbbb = find_if_close(cnt1,cnt2,50)
                if dist == True:
                    val = min(status[i],status[x])
                    status[x] = status[i] = val
                else:
                    if status[x]==status[i]:
                        status[x] = i+1
    sizeold=len(linearr)
    rectArr=[]
    if(len(status>0)):
        oldUnified=list(unified)
        unified=[]
        maximum = int(status.max())+1
        for i in range(maximum):
            pos = np.where(status==i)[0]
            if pos.size != 0:
                cont = np.vstack(contours[i] for i in pos)
                x,y,w,h = cv2.boundingRect(cont)
                #cv2.rectangle(frame,(x,y),(x+w,y+h),(150,60,60),2)
                rectArr.append((x,y,w,h))
    if(len(rectArr)>50):
        continue
    if(perFrame==True):
        for ind,it in enumerate(rectArr):
            print(len(baseframe[0]))
            x,y,w,h=it
            deltaX=40
            deltaY=40
            if(y-deltaY<0 or x-deltaX<0 or x+w+deltaX>len(baseframe[0]) or y+h+deltaY>len(baseframe)):
                continue
            frame=baseframe.copy()
            cv2.rectangle(frame,(x-deltaX,y-deltaY),(x+w+deltaX,y+h+deltaY),(0,0,255),2)
            roi=baseframe[y-deltaY:y+h+deltaY,x-deltaX:x+w+deltaX]
            cv2.imshow('asdasdasff',frame)
            key = cv2.waitKey(0)
            if(key==ord('a')):
                saveMatInDir(roi,'C:\\Users\\dev-user\\Documents\\marking\\notBallDir\\',str(iii)+'_'+str(ind)+'.jpg')
            if(key==ord('b')):
                saveMatInDir(roi,'C:\\Users\\dev-user\\Documents\\marking\\ballDir\\',str(iii)+'_'+str(ind)+'.jpg')
            if(key==27):
                break
            if(key==32):
                perFrame=False
            #cv2.rectangle(frame,(x,y),(x+w,y+h),(150,60,60),2)
    
    #cv2.drawContours(black,unified,-1, (255,255,255), -1)
    #cv2.drawContours(frame,unified,-1, (0,255,0), 2)
   
    cv2.imshow('asdasdf',fgmask)
    cv2.imshow('asdasdff',frame)
    
    if(len(rectArr)>0 and perFrame==True):
        continue
    delay = 1
    k = cv2.waitKey(delay)
    if(k!=-1):
        print (k)
    if k == 27:
        break
    if k == 112:
        cv2.imwrite('pic{:>05}.jpg'.format(iii), res)
    if k == 32 or perFrame==True:
        k = cv2.waitKey(0)
        if(k==32):
            perFrame=False
            continue
        if k == 112:
            cv2.imwrite('pic{:>05}.jpg'.format(iii), res)
            cv2.imwrite('picr{:>05}.jpg'.format(iii), res1)
        if(k == 2555904): 
            perFrame=True
            continue
    
        
    

cap.release()
cv2.destroyAllWindows()

32
1280
1280
1280
1280
1280
1280
1280
1280
1280
1280
1280
1280
1280
1280
1280
1280
1280
1280
1280
1280
1280
1280
1280
1280
1280
1280
1280
1280
1280
1280
1280
1280
1280
1280
1280
1280
1280
1280
1280
1280
1280
1280
1280
1280
1280
1280
1280
1280
27


In [22]:
arr=[]
x,y,z,h=1,2,3,4
arr.append((x,y,z,h))

(a,na,n,sd)=arr[0]
print(n)

3
